In [136]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
df = pd.read_csv('Data_Marketing_Customer_Analysis_Round3.csv')
df = pd.DataFrame(df)
df

,region,customer_lifetime_value,response,coverage,education,effective_to_date,month,employment_status,gender,income,...,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,central,4809,no,basic,college,2/18/11,feb,employed,m,48029,...,52,0,9,corporate auto,corporate l3,offer3,agent,292,four-door car,medsize
1,west region,2228,no,basic,college,1/18/11,jan,unemployed,f,92260,...,26,0,1,personal auto,personal l3,offer4,call center,744,four-door car,medsize
2,east,14947,no,basic,bachelor,2/10/11,feb,employed,m,22139,...,31,0,2,personal auto,personal l3,offer3,call center,480,suv,medsize
3,north west,22332,yes,extended,college,1/11/11,jan,employed,m,49078,...,3,0,2,corporate auto,corporate l3,offer2,branch,484,four-door car,medsize
4,north west,9025,no,premium,bachelor,1/17/11,jan,medical leave,f,23675,...,31,0,7,personal auto,personal l2,offer1,branch,707,four-door car,medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10684,central,15563,no,premium,bachelor,1/19/11,jan,unemployed,f,61541,...,40,0,7,personal auto,personal l1,offer3,web,1214,luxury car,medsize
10685,north west,5259,no,basic,college,1/6/11,jan,employed,f,61146,...,68,0,6,personal auto,personal l3,offer2,branch,273,four-door car,medsize
10686,central,23893,no,extended,bachelor,2/6/11,feb,employed,f,39837,...,63,0,2,corporate auto,corporate l3,offer1,web,381,luxury suv,medsize
10687,west region,11971,no,premium,college,2/13/11,feb,employed,f,64195,...,27,4,6,personal auto,personal l1,offer1,branch,618,suv,medsize


In [137]:
TT_RATIO = 0.3
RAND_STATE = 40
x = df.drop('total_claim_amount', axis=1)
y = df.total_claim_amount
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=TT_RATIO, random_state=RAND_STATE)
x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)
numericalX = x.select_dtypes(np.number)
categoricalx = x.select_dtypes(np.object)

C:\Users\pedro\AppData\Local\Temp\ipykernel_12640\3405005441.py:9: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  categoricalx = x.select_dtypes(np.object)


In [138]:
pt = PowerTransformer()
ct = ColumnTransformer([("pt", pt, list(numericalX.columns))],
                        remainder='drop',verbose_feature_names_out=True,verbose=True).fit(x_train)
x_train_ct = pd.DataFrame(ct.transform(x_train),columns=ct.get_feature_names_out())
x_test_ct = pd.DataFrame(ct.transform(x_test),columns=ct.get_feature_names_out())

[ColumnTransformer] ............ (1 of 1) Processing pt, total=   0.1s


In [139]:
x_train = pd.DataFrame(x_train_ct)
x_test_ct

,pt__customer_lifetime_value,pt__income,pt__monthly_premium_auto,pt__months_since_last_claim,pt__months_since_policy_inception,pt__number_of_open_complaints,pt__number_of_policies
0,0.050730,-1.356158,-0.381873,0.150225,0.493307,1.888107,0.882125
1,0.376214,0.786262,0.399984,1.335482,0.681554,-0.516421,1.134298
2,-0.052632,0.296556,-1.440994,-0.040921,-1.653895,2.003352,-0.061278
3,-0.256654,1.462037,-1.254575,0.417890,-0.286896,2.003352,1.577131
4,-0.221355,0.858966,1.216644,1.603955,0.099597,-0.516421,-1.146658
...,...,...,...,...,...,...,...
3202,-1.336125,1.492578,-0.633343,0.746837,-2.000985,1.998886,-1.146658
3203,0.052056,-0.718567,-0.213581,0.241767,-0.667832,1.888107,1.670156
3204,0.730183,-0.336876,1.127574,1.195427,1.642058,-0.516421,0.517797
3205,-0.075114,-0.455039,1.411875,0.241767,0.167012,-0.516421,-1.146658


In [140]:
x_train_const_ct = sm.add_constant(x_train_ct.to_numpy()) # adding a constant

model = sm.OLS(y_train, x_train_const_ct).fit()
predictions_train = model.predict(x_train_const_ct)

x_test_const_ct = sm.add_constant(x_test_ct) # adding a constant
predictions_test = model.predict(x_test_const_ct)
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:     total_claim_amount   R-squared:                       0.319
Model:                            OLS   Adj. R-squared:                  0.318
Method:                 Least Squares   F-statistic:                     500.0
Date:                Mon, 20 Feb 2023   Prob (F-statistic):               0.00
Time:                        17:59:51   Log-Likelihood:                -51703.
No. Observations:                7482   AIC:                         1.034e+05
Df Residuals:                    7474   BIC:                         1.035e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        435.4237      2.806    155.166      0.0

In [141]:
##OLS
model=LinearRegression()    # model
model.fit(x_train, y_train)   # model train
print(f"{model.__class__.__name__}: Train -> {model.score(x_train, y_train)}, Test -> {model.score(x_test_ct, y_test)}")

LinearRegression: Train -> 0.3189432948161396, Test -> 0.28959643027960713


In [142]:
##Lasso
from sklearn.linear_model import Lasso,Ridge,ElasticNet, LinearRegression
model2=Lasso(alpha=20)

model2.fit(x_train, y_train)
print(f"{model2.__class__.__name__}: Train -> {model2.score(x_train, y_train)}, Test -> {model2.score(x_test_ct, y_test)}")

Lasso: Train -> 0.30920808334243477, Test -> 0.2865617828845314


In [143]:
##Ridge
model3=Ridge(alpha=100)
model3.fit(x_train, y_train)
print(f"{model3.__class__.__name__}: Train -> {model3.score(x_train, y_train)}, Test -> {model3.score(x_test_ct, y_test)}")

Ridge: Train -> 0.31886427755830926, Test -> 0.2900517076961817


In [144]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE  ## recursive feature elemination technique

In [145]:
lm = LinearRegression()

selector = RFE(lm, n_features_to_select= 5, step = 1, verbose = 1) # Step is how many features to add or drop everytime
selector.fit(x_train, y_train)

kept_features = selector.get_support(indices = True) #returns an array of integers corresponding to nonremoved features
kept_features = list(x_train.iloc[:,kept_features].columns)

x_train = selector.transform(x_train)
x_test  = selector.transform(x_test_ct)
y_train = elector.transform(x_test_ct)

x_train = pd.DataFrame(x_train, columns=kept_features)
x_test  = pd.DataFrame(x_test_ct, columns=kept_features)

print("Final selected features: ")
display(x_train)

Fitting estimator with 7 features.
Fitting estimator with 6 features.
Final selected features: 


,pt__customer_lifetime_value,pt__income,pt__monthly_premium_auto,pt__number_of_open_complaints,pt__number_of_policies
0,-0.377775,1.030642,1.028939,-0.516421,-1.146658
1,-0.548744,0.043272,0.871857,-0.516421,-1.146658
2,1.982802,-0.656078,2.105524,-0.516421,-0.061278
3,-0.622323,0.454820,0.743503,-0.516421,-1.146658
4,0.505610,1.107371,-0.701676,-0.516421,-0.061278
...,...,...,...,...,...
7477,0.512512,1.385679,0.715980,-0.516421,0.517797
7478,1.129977,0.339074,-1.081895,-0.516421,-0.061278
7479,-0.254950,0.509077,-1.345973,-0.516421,0.517797
7480,-0.300349,0.244883,-1.539842,-0.516421,1.134298


In [146]:
##OLS
model=LinearRegression()    # model
model.fit(x_train, y_train)   # model train
print(f"{model.__class__.__name__}: Train -> {model.score(x_train, y_train)})

SyntaxError: EOL while scanning string literal (500767441.py, line 4)